In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# ! pip install gym

In [3]:
from gym.spaces import Discrete
from gym.spaces import Tuple
from gym.spaces.box import Box

In [4]:
throttle_brake = Discrete(3)  # -1 brake, 0 keep, 1 throttle
steering = Discrete(3)
action_space = Tuple([throttle_brake,steering])

N = 4 # FIXME not hard code
F = 6 # FIXME not hard code
state_space = Box(low=-np.inf, high=np.inf, shape=(N,F), dtype=np.float32)

In [6]:
from tensorflow.keras.layers import Input, Dense, Lambda, Multiply, Reshape, Flatten, Masking, LSTM, Concatenate
from tensorflow.keras.models import Model



In [14]:
CAV_state_input = Input(name="cav_input",shape=(6,))
CAV_action_input = Input(name="cav_action",shape=(3,)) #throttle, brake, steering
input_concate = Concatenate(axis=-1)([CAV_state_input,CAV_action_input])
x = Dense(32,activation='relu',name="CAV_dynamics_1")(input_concate)
x = Dense(16,activation='relu',name="CAV_dynamics_2")(x)
out = Dense(6,activation=None)(x)
CAV_dynamics = Model(inputs=[CAV_state_input,CAV_action_input], outputs=out)
CAV_dynamics.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cav_input (InputLayer)          [(None, 6)]          0                                            
__________________________________________________________________________________________________
cav_action (InputLayer)         [(None, 3)]          0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 9)            0           cav_input[0][0]                  
                                                                 cav_action[0][0]                 
__________________________________________________________________________________________________
CAV_dynamics_1 (Dense)          (None, 32)           320         concatenate_7[0][0]        

In [17]:
HDV_state_input = Input(name="hdv_input",shape=(6,))
x = Dense(32,activation='relu',name="HDV_dynamics_1")(HDV_state_input)
x = Dense(16,activation='relu',name="HDV_dynamics_2")(x)
out = Dense(6,activation=None)(x)
HDV_dynamics = Model(inputs=HDV_state_input, outputs=out)
HDV_dynamics.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hdv_input (InputLayer)       [(None, 6)]               0         
_________________________________________________________________
HDV_dynamics_1 (Dense)       (None, 32)                224       
_________________________________________________________________
HDV_dynamics_2 (Dense)       (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 102       
Total params: 854
Trainable params: 854
Non-trainable params: 0
_________________________________________________________________


# Reverse the control commands

In [7]:
import glob
import pickle

In [10]:
!pwd

/Users/jiqiandong/Desktop/OneDrive - purdue.edu/work2/carla_rl


In [12]:
with open("./control_details/LHDV.p", 'rb') as f:
    commands = pickle.load(f)

In [16]:
commands_reversed = []
for command in commands:
    c = command.copy()
    c['steer'] = -command['steer']
    commands_reversed.append(c)

In [17]:
commands_reversed[50]

{'throttle': 0.0,
 'steer': -0.0,
 'brake': 0.0,
 'hand_brake': 0,
 'reverse': False,
 'manual_gear_shift': True,
 'gear': 0}

In [18]:
commands[50]

{'throttle': 0.0,
 'steer': 0.0,
 'brake': 0.0,
 'hand_brake': 0,
 'reverse': False,
 'manual_gear_shift': True,
 'gear': 0}

In [19]:
import random

In [31]:
random.choice([0,1])

0

In [32]:
with open("./control_details/LHDV_right.p", 'wb') as f:
    pickle.dump(commands_reversed,f)